<a href="https://colab.research.google.com/github/Sam-ai904/Huatai-Model/blob/main/%E9%81%97%E4%BC%A0%E8%A7%84%E5%88%92_%E7%A0%94%E6%8A%A5%E7%89%88%2B%E9%AB%98%E6%80%A7%E8%83%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install tushare
!pip install bottleneck
import tushare as ts
import pandas as pd
import numpy as np
import random
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
import bottleneck as bn


In [2]:
# 设置Tushare API Token并初始化
ts.set_token('2876ea85cb005fb5fa17c809a98174f2d5aae8b1f830110a5ead6211')
pro = ts.pro_api()


In [3]:
# 获取沪深300成分股前50
def get_hs300_top50(start_date , end_date):
    try:
        df = pro.index_weight(index_code='000001.SH', start_date= start_date, end_date= end_date)
        if df.empty:
            raise ValueError("未获取到沪深300成分股数据")

        df = df.sort_values('weight', ascending=False)
        stock_list = df['con_code'].unique()[:100].tolist()
        print(f"成功获取沪深300前50只股票: {stock_list[:6]}...（共 {len(stock_list)} 只）")
        return stock_list

    except Exception as e:
        print(f"获取沪深300成分股失败: {e}")
        return []

# 获取股票日频数据
def get_data(start_date, end_date, stock_list):
    df_list = []
    for stock in stock_list:
        try:
            temp_df = pro.daily(
                ts_code=stock,
                start_date=start_date,
                end_date=end_date,
                fields='ts_code,trade_date,open,close,high,low,vol,pct_chg'
            )
            if not temp_df.empty:
                # print(f"股票 {stock} 在 {start_date} 至 {end_date} 获取到 {len(temp_df)} 条数据")
                df_list.append(temp_df)
            else:
                print(f"股票 {stock} 在 {start_date} 至 {end_date} 无数据")
        except Exception as e:
            print(f"获取股票 {stock} 数据失败: {e}")

    if not df_list:
        print(f"时间范围 {start_date} 至 {end_date} 无任何股票数据")
        return pd.DataFrame()

    try:
        df = pd.concat(df_list)
        df.rename(columns={'vol': 'volume', 'pct_chg': 'return'}, inplace=True)
        df['return'] = df['return'] / 100
        df['trade_date'] = pd.to_datetime(df['trade_date'])

        # 检查并移除重复的 trade_date 和 ts_code 组合
        duplicates = df.duplicated(subset=['trade_date', 'ts_code'], keep=False)
        if duplicates.any():
            print(f"警告: 发现重复数据，共 {duplicates.sum()} 条，自动保留最后一条")
            df = df.drop_duplicates(subset=['trade_date', 'ts_code'], keep='last')

        # 重塑数据
        df_pivot = df.pivot(index='trade_date', columns='ts_code')

        # 检查收益率数据
        if 'return' in df_pivot:
            return_stats = df_pivot['return'].describe()
            print(f"收益率统计: {return_stats}")
            # 过滤收益率全为 NaN 或常数的股票
            valid_stocks = return_stats.loc['std'] > 0
            valid_stocks = valid_stocks[valid_stocks].index.tolist()
            if not valid_stocks:
                print("所有股票的收益率均为常数或 NaN，无法继续")
                return pd.DataFrame()
            df_pivot = df_pivot.loc[:, df_pivot.columns.get_level_values(1).isin(valid_stocks)]

        return df_pivot
    except Exception as e:
        print(f"数据合并失败: {e}")
        return pd.DataFrame()

In [4]:
# 初始化种群
def initialize_population(size, function_list):
    population = []
    for _ in range(size):
        formula = random.choice(function_list)
        population.append(formula)
    return population


In [5]:
# 修改后的 calculate_fitness 函数
def calculate_fitness(formula, data):
    try:
        factor_values = eval(formula, {'np': np, 'bn': bn}, {'data': data})
        # 将 factor_values 转换为 numpy 数组并展平
        if isinstance(factor_values, pd.DataFrame):
            factor_values_flat = factor_values.values.flatten()
        else:
            factor_values_flat = factor_values.flatten()
        # 将 returns 转换为 numpy 数组并展平
        returns_flat = data['return'].values.flatten()

        # 检查数据
        # print(f"因子公式: {formula}")
        # print(f"因子值形状: {factor_values.shape if isinstance(factor_values, (pd.DataFrame, np.ndarray)) else 'N/A'}, 展平后: {factor_values_flat.shape}")
        print(f"因子值统计: min={np.nanmin(factor_values_flat):.4f}, max={np.nanmax(factor_values_flat):.4f}, std={np.nanstd(factor_values_flat):.4f}")
        # print(f"因子值 NaN 比例: {np.isnan(factor_values_flat).mean():.4f}")
        print(f"收益率形状: {data['return'].shape}, 展平后: {returns_flat.shape}")
        print(f"收益率统计: min={np.nanmin(returns_flat):.4f}, max={np.nanmax(returns_flat):.4f}, std={np.nanstd(returns_flat):.4f}")
        # print(f"收益率 NaN 比例: {np.isnan(returns_flat).mean():.4f}")

        mask = ~(np.isnan(factor_values_flat) | np.isnan(returns_flat))
        if mask.sum() < 2:
            print("有效数据点少于 2，无法计算相关性")
            return -1

        # 检查是否为常数
        if np.nanstd(factor_values_flat[mask]) == 0 or np.nanstd(returns_flat[mask]) == 0:
            print("因子值或收益率是常数，无法计算相关性")
            return -1

        corr, _ = spearmanr(factor_values_flat[mask], returns_flat[mask])
        print(f"RankIC(corr): {corr:.4f}")
        return corr if not np.isnan(corr) else -1
    except Exception as e:
        print(f"计算适应度失败: {e}")
        return -1

In [6]:
def evolve_population(population, data, generations):
    for gen in range(generations):
        fitness_scores = [(formula, calculate_fitness(formula, data)) for formula in population]
        # 打印 fitness_scores 的统计信息
        scores = [score for _, score in fitness_scores if not np.isnan(score)]
        print(f"第 {gen+1} 代，适应度统计: min={min(scores) if scores else 'N/A'}, max={max(scores) if scores else 'N/A'}, mean={np.mean(scores) if scores else 'N/A'}")

        fitness_scores = sorted(fitness_scores, key=lambda x: x[1], reverse=True)[:int(len(fitness_scores)*0.6)]
        # 暂时移除筛选条件，确保种群不为空
        population = [item[0] for item in fitness_scores if item[1] > 0.015]

        print(f"第 {gen+1} 代，筛选后种群大小: {len(population)}")

        if not population:
            print("种群为空，停止进化，可能是因子公式无效或数据问题")
            return []

        new_population = []
        while len(new_population) < len(population):
            parent1, parent2 = random.sample(population, 2)
            new_formula = f"({parent1}) + ({parent2})"
            if random.random() < 0.1:
                operations = ['+', '*', '-']
                new_formula = new_formula.replace('+', random.choice(operations))
            new_population.append(new_formula)
        population = new_population[:len(population)]
    return population

In [7]:
# 计算残差收益率
def calculate_residual_return(data, factor_pool):
    if not factor_pool:
        # 展平 data['return'] 为一维数组
        residual = data['return'].values.flatten()
        print(f"初始残差收益率形状: {data['return'].shape}, 展平后: {residual.shape}")
        return pd.Series(residual)
    try:
        X = np.column_stack([eval(formula, {'np': np, 'bn': bn}, {'data': data}) for formula in factor_pool])
        y = data['return'].values.flatten()
        mask = ~(np.isnan(X).any(axis=1) | np.isnan(y))
        if mask.sum() < 2:
            print("残差计算数据点不足，返回原始收益率")
            return pd.Series(y)
        lr = LinearRegression()
        lr.fit(X[mask], y[mask])
        residuals = y - lr.predict(X)
        print(f"残差计算后形状: {residuals.shape}")
        return pd.Series(residuals)
    except Exception as e:
        print(f"残差计算错误: {e}")
        return pd.Series(data['return'].values.flatten())


In [8]:
# 修改后的 rolling_factor_extraction 函数（更新了 function_list）
def rolling_factor_extraction(start_date, end_date, interval_years=2):
    stock_list = get_hs300_top50(start_date, end_date)
    if not stock_list:
        print("无法获取股票列表，退出")
        return []

    current_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    factor_pool = []
    # 更新因子公式，增加复杂性和多样性
    function_list = [
        "data['open'] / (data['close'] + 1e-10)",  # 价格比率
        "(data['high'] - data['low']) / (data['close'] + 1e-10)",  # 波动性
        "bn.move_mean(data['volume'], window=5) / (bn.move_mean(data['volume'], window=20) + 1e-10)",  # 成交量均值比率
        "np.log(data['close'] + 1) - np.log(bn.move_mean(data['close'], window=5) + 1)",  # 价格对数差
        "bn.nanrankdata(data['close'].diff(1), axis=0) / (data['close'] + 1e-10)",  # 价格变化排名
        "(data['close'] - bn.move_mean(data['close'], window=10)) / bn.move_std(data['close'], window=10)"  # 标准化价格偏差
    ]

    # 设置最早数据日期
    earliest_date = pd.to_datetime('20150101')

    while current_date < end_date:
        sample_start = (current_date - pd.DateOffset(years=2))
        if sample_start < earliest_date:
            sample_start = earliest_date
        sample_start = sample_start.strftime('%Y%m%d')
        sample_end = current_date.strftime('%Y%m%d')

        print(f"正在处理窗口: {sample_start} 至 {sample_end}")
        data = get_data(sample_start, sample_end, stock_list)

        if data.empty:
            print(f"无数据: {sample_start} 至 {sample_end}")
            current_date += pd.DateOffset(years=interval_years)
            continue

        population_size = 200
        generations = 2
        population = initialize_population(population_size, function_list)

        try:
            residual_return = calculate_residual_return(data, factor_pool)
            # 确保 residual_return 是一维数组
            if isinstance(residual_return, pd.Series):
                residual_return_flat = residual_return.values
            else:
                residual_return_flat = residual_return.flatten()
            print(f"残差收益率统计: min={np.nanmin(residual_return_flat):.4f}, max={np.nanmax(residual_return_flat):.4f}, std={np.nanstd(residual_return_flat):.4f}")

            print("开始进化种群...")
            final_population = evolve_population(population, data, generations)
            print(f"最终种群大小: {len(final_population)}")

            for formula in final_population:
                try:
                    factor_values = eval(formula, {'np': np, 'bn': bn}, {'data': data})
                    # 展平二维数组，确保顺序一致
                    if isinstance(factor_values, pd.DataFrame):
                        factor_values_flat = factor_values.values.flatten()
                    else:
                        factor_values_flat = factor_values.flatten()
                    mask = ~(np.isnan(factor_values_flat) | np.isnan(residual_return_flat))
                    if mask.sum() < 2:
                        print(f"因子 {formula} 有效数据点少于 2，跳过")
                        continue

                    corr, _ = spearmanr(factor_values_flat[mask], residual_return_flat[mask])
                    # print(f"因子 {formula} 与残差的相关性: {corr:.4f}")

                    if abs(corr) < 0.7 and not np.isnan(corr):
                        factor_pool.append(formula)
                        # print(f"添加因子: {formula}, 相关性: {corr:.4f}")
                except Exception as e:
                    print(f"因子 {formula} 处理失败: {e}")
                    continue
        except Exception as e:
            print(f"处理错误: {e}")
            continue

        current_date += pd.DateOffset(years=interval_years)
        factor_pool = factor_pool[-100:]

    return factor_pool

In [9]:
def main():
    start_date = '20180101'
    end_date = '20240101'
    print(f"因子挖掘范围: {start_date} 至 {end_date}")
    rolling_factors = rolling_factor_extraction(start_date, end_date)
    print("挖掘的因子公式:")
    if not rolling_factors:
        print("因子池为空，可能是因子公式无效或数据问题，请检查日志")
    for factor in rolling_factors:
        print(f'{factor}\n')

if __name__ == "__main__":
    main()

因子挖掘范围: 20200101 至 20240101
成功获取沪深300前50只股票: ['600519.SH', '300750.SZ', '601318.SH', '600036.SH', '000858.SZ', '000333.SZ']...（共 50 只）
正在处理窗口: 20180101 至 20200101
股票 601816.SH 在 20180101 至 20200101 无数据
股票 688981.SH 在 20180101 至 20200101 无数据
收益率统计: ts_code   000001.SZ   000002.SZ   000063.SZ   000333.SZ   000568.SZ  \
count    487.000000  487.000000  448.000000  447.000000  487.000000   
mean       0.000706    0.000495    0.000543    0.000452    0.000985   
std        0.020751    0.022967    0.034756    0.021616    0.026351   
min       -0.077100   -0.092593   -0.100200   -0.100000   -0.094761   
25%       -0.010722   -0.011852   -0.015126   -0.011199   -0.015189   
50%       -0.000800   -0.001078    0.000421    0.000374    0.000200   
75%        0.011868    0.012040    0.016608    0.011027    0.015161   
max        0.087522    0.090300    0.100112    0.086189    0.100071   

ts_code   000625.SZ   000651.SZ   000725.SZ   000792.SZ   000858.SZ  ...  \
count    487.000000  481.000000  487